#A Music Recommendation System, a product of Data Science, is designed to help users explore new and suitable music content by analyzing their preferences and listening habits. These personalized recommendations play a vital role in the digital music realm, empowering platforms such as Spotify and Apple Music to deliver tailored and immersive experiences to their audience.

##What is the Functionality of a Music Recommendation System?

###Music Recommendation Systems rely on sophisticated algorithms that analyze extensive datasets encompassing users’ musical interactions, including their listening history, favored tracks, skipped songs, and user-provided preferences like ratings or feedback. These datasets play a pivotal role in constructing detailed user profiles, capturing individual tastes and preferences accurately.

###Initially, the system employs various data preprocessing techniques to cleanse and organize the data effectively. Following this phase, recommendation algorithms such as collaborative filtering, content-based filtering, and hybrid approaches are utilized to generate personalized music suggestions.

###As users engage with the system over time, it continuously gathers additional data, refining and updating their profiles dynamically. Consequently, the recommendations become increasingly accurate and tailored to the user’s evolving musical tastes and preferences.

#What does the Spotify API offer, and how can it be utilized to create a Music Recommendation System?

###The Spotify API, crafted by Spotify developers, furnishes a framework for interacting with Spotify's extensive music library and retrieving music-related data. By leveraging this API, developers gain access to a plethora of information, including tracks, albums, artists, playlists, user profiles, and listening history. This access empowers developers to create inventive applications and services that seamlessly integrate with the Spotify platform.

###To construct a Music Recommendation System utilizing the Spotify API, one must first acquire real-time music data from Spotify. This necessitates the acquisition of credentials from Spotify through a developer account, enabling access to their data.

#A music recommendation system implemented in Python

In [33]:
import requests
import base64


CLIENT_ID = '4af3cb7b9d36457aa677b34379c27f7a'
CLIENT_SECRET = '3a9b01621e4340179be72673d53259ba'

# Base64 encode the client ID and client secret
client_credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"
client_credentials_base64 = base64.b64encode(client_credentials.encode())

# Request the access token
token_url = 'https://accounts.spotify.com/api/token'
headers = {
    'Authorization': f'Basic {client_credentials_base64.decode()}'
}
data = {
    'grant_type': 'client_credentials'
}
response = requests.post(token_url, data=data, headers=headers)

if response.status_code == 200:
    access_token = response.json()['access_token']
    print("The access token has been successfully acquired.")
else:
    print("An error occurred while obtaining the access token.")
    exit()

The access token has been successfully acquired.


##In the provided code snippet, the variables CLIENT_ID and CLIENT_SECRET hold my unique credentials, which must be replaced with your own obtained from Spotify's developer dashboard upon registering your application. These credentials are essential for authenticating the application with the Spotify API. The CLIENT_ID identifies the application, while the CLIENT_SECRET serves as a confidential key for authentication.

##The client credentials are combined into the client_credentials variable, separated by a colon (:), and then encoded using Base64 encoding to ensure secure transmission. Subsequently, a POST request is sent to the token_url (https://accounts.spotify.com/api/token) with the client credentials included in the Authorization header. The grant_type parameter is set to 'client_credentials' to indicate that the application is requesting an access token using the client credentials flow.

##Upon successful authentication, an access token is obtained, enabling the application to make authorized requests to retrieve music-related data from the Spotify API, including tracks, albums, artists, and user information. This access token is integral for developing a music recommendation system using the Spotify API and Python.

In [34]:
!pip install spotipy

In [35]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth

def get_trending_playlist_data(playlist_id, access_token):
    # Set up Spotipy with the access token
    sp = spotipy.Spotify(auth=access_token)

    # Get the tracks from the playlist
    playlist_tracks = sp.playlist_tracks(playlist_id, fields='items(track(id, name, artists, album(id, name)))')

    # Extract relevant information and store in a list of dictionaries
    music_data = []
    for track_info in playlist_tracks['items']:
        track = track_info['track']
        track_name = track['name']
        artists = ', '.join([artist['name'] for artist in track['artists']])
        album_name = track['album']['name']
        album_id = track['album']['id']
        track_id = track['id']

        # Get audio features for the track
        audio_features = sp.audio_features(track_id)[0] if track_id != 'Not available' else None

        # Get release date of the album
        try:
            album_info = sp.album(album_id) if album_id != 'Not available' else None
            release_date = album_info['release_date'] if album_info else None
        except:
            release_date = None

        # Get popularity of the track
        try:
            track_info = sp.track(track_id) if track_id != 'Not available' else None
            popularity = track_info['popularity'] if track_info else None
        except:
            popularity = None


        track_data = {
            'Track Name': track_name,
            'Artists': artists,
            'Album Name': album_name,
            'Album ID': album_id,
            'Track ID': track_id,
            'Popularity': popularity,
            'Release Date': release_date,
            'Duration (ms)': audio_features['duration_ms'] if audio_features else None,
            'Explicit': track_info.get('explicit', None),
            'External URLs': track_info.get('external_urls', {}).get('spotify', None),
            'Danceability': audio_features['danceability'] if audio_features else None,
            'Energy': audio_features['energy'] if audio_features else None,
            'Key': audio_features['key'] if audio_features else None,
            'Loudness': audio_features['loudness'] if audio_features else None,
            'Mode': audio_features['mode'] if audio_features else None,
            'Speechiness': audio_features['speechiness'] if audio_features else None,
            'Acousticness': audio_features['acousticness'] if audio_features else None,
            'Instrumentalness': audio_features['instrumentalness'] if audio_features else None,
            'Liveness': audio_features['liveness'] if audio_features else None,
            'Valence': audio_features['valence'] if audio_features else None,
            'Tempo': audio_features['tempo'] if audio_features else None,

        }

        music_data.append(track_data)

    # Create a pandas DataFrame from the list of dictionaries
    df = pd.DataFrame(music_data)

    return df

##The function starts by initializing the Spotipy client using the provided access_token, which acts as the authentication token for accessing the Spotify Web API. This token authorizes the function to make requests to access Spotify's resources. Using the Spotipy client, the function retrieves information about the tracks within the specified playlist, identified by its playlist_id. The sp.playlist_tracks method is employed for this purpose, with the fields parameter specifying the desired track information such as track ID, name, artists, album ID, and album name.

##Subsequently, the function extracts pertinent details from the retrieved playlist tracks and organizes them into a list of dictionaries named music_data. For each track in the playlist, the function captures data like track name, artists (combined into a single string), album name, album ID, track ID, and popularity. Additionally, the function utilizes the sp.audio_features method to obtain audio features for each track. These features encompass attributes such as danceability, energy, key, loudness, speechiness, acousticness, instrumentalness, liveness, valence, tempo, and more, offering insights into the characteristics of each track.

#Now, let's explore how we can utilize the function to gather music data from any playlist available on Spotify:

In [36]:
#37i9dQZF1DX5cZuAHLNjGz ---- Playlist id

playlist_id = '37i9dQZF1DX5cZuAHLNjGz'

# Call the function to get the music data from the playlist and store it in a DataFrame
music_df = get_trending_playlist_data(playlist_id, access_token)

# Display the DataFrame
music_df

,Track Name,Artists,Album Name,Album ID,Track ID,Popularity,Release Date,Duration (ms),Explicit,External URLs,...,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
0,Goin' Off,"Karan Aujla, Mxrci",Goin' Off,3uXGbWKTUkC14RmbgjTQlM,3txE0SZnwamBhYB2ZQtHwU,67,2024-05-09,163000,False,https://open.spotify.com/track/3txE0SZnwamBhYB...,...,0.897,2,-4.002,1,0.2090,0.0873,0.000000,0.1730,0.381,84.125
1,MVP,Shubh,MVP,5oCCfkVCpkSUhBUXe0pyJF,5BQePe9rcCJozHK3oPayYk,62,2024-05-10,196898,True,https://open.spotify.com/track/5BQePe9rcCJozHK...,...,0.572,7,-8.163,0,0.0743,0.3910,0.004280,0.3180,0.644,88.041
2,California Love,"Cheema Y, Gur Sidhu",ANYWAY,64MZHXV7IglO8LE8KplSRw,610keNiNVTIkYD0CqeSerg,75,2023-01-30,175687,True,https://open.spotify.com/track/610keNiNVTIkYD0...,...,0.583,1,-6.145,0,0.2950,0.2060,0.000001,0.1170,0.458,178.113
3,8 ASLE,"Sukha, Chani Nattan, Prodgk, Gurlez Akhtar",UNDISPUTED,5jwxgwhfEuWqBfKYPHaC34,6wkHR8cU4INbp145hngbQO,74,2023-11-17,161000,True,https://open.spotify.com/track/6wkHR8cU4INbp14...,...,0.684,10,-6.113,0,0.1480,0.0977,0.000000,0.1800,0.483,100.042
4,"Naina (From ""Crew"")","Diljit Dosanjh, Badshah, Raj Ranjodh","Naina (From ""Crew"")",4mGz0G0d2mqGmaFc67MEEm,1eZefeDb8uOsjvcbl1fJrG,81,2024-03-05,180000,False,https://open.spotify.com/track/1eZefeDb8uOsjvc...,...,0.715,0,-5.141,1,0.1870,0.1760,0.000000,0.0935,0.542,96.980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Kina Chir,The PropheC,The Lifestyle,1zIvQYNdDnl4f7iF2s6HHj,6XPmY4hIb1RGRyNuq0UBsl,67,2016-12-29,216027,False,https://open.spotify.com/track/6XPmY4hIb1RGRyN...,...,0.431,11,-8.260,0,0.0293,0.0955,0.000000,0.0880,0.365,117.919
96,Parshawan,Harnoor,Parshawan,6w6MwWpj1oR3lKpQPc8caY,6IaHIJYxIG8xbosaRr25AC,1,2021-06-10,168000,False,https://open.spotify.com/track/6IaHIJYxIG8xbos...,...,0.440,1,-7.422,0,0.0610,0.5590,0.000000,0.1050,0.430,90.006
97,Ykwim,"Karan Aujla, KR$NA, Mehar Vaani",Ykwim,14XBYP5RcavbvSREiaQ17V,6qS40yL7kXVoChI8SSCQIQ,68,2022-01-18,194857,False,https://open.spotify.com/track/6qS40yL7kXVoChI...,...,0.575,5,-7.971,0,0.1270,0.0361,0.000007,0.4570,0.655,105.015
98,Love Sick,"Sidhu Moose Wala, AR Paisley",No Name,57SIarHHgfby4AvV6ftCB5,1skGwRjc7wYY70PJCAkKMr,64,2022-04-25,232258,False,https://open.spotify.com/track/1skGwRjc7wYY70P...,...,0.543,4,-8.667,0,0.2090,0.1010,0.000000,0.1780,0.548,124.018


In [37]:
print(music_df.isnull().sum())

Track Name          0
Artists             0
Album Name          0
Album ID            0
Track ID            0
Popularity          0
Release Date        0
Duration (ms)       0
Explicit            0
External URLs       0
Danceability        0
Energy              0
Key                 0
Loudness            0
Mode                0
Speechiness         0
Acousticness        0
Instrumentalness    0
Liveness            0
Valence             0
Tempo               0
dtype: int64


##Moving forward, let's proceed with constructing a music recommendation system using Python. Let's begin by importing the required Python libraries:

In [38]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity

data = music_df

###When delivering music recommendations to users, prioritizing the latest releases is crucial. To achieve this, we should assign greater significance to the most recent releases in our recommendations. Let's create a function to address this requirement.

In [39]:
# Function to calculate weighted popularity scores based on release date
def calculate_weighted_popularity(release_date):
    # Convert the release date to datetime object
    release_date = datetime.strptime(release_date, '%Y-%m-%d')

    # Calculate the time span between release date and today's date
    time_span = datetime.now() - release_date

    # Calculate the weighted popularity score based on time span (e.g., more recent releases have higher weight)
    weight = 1 / (time_span.days + 1)

    return weight

###The provided function accepts the release date of a music track in the format 'YYYY-MM-DD' as input. It utilizes the datetime.strptime function from the Python datetime module to convert the release date string into a datetime object. This conversion enables the function to perform various arithmetic operations involving dates. Subsequently, it computes the duration between the release date of the track and the current date (today's date) by subtracting the release date from datetime.now(). This computation yields a timedelta object representing the time gap between the two dates.

###The weighted popularity score is determined based on the duration. It utilizes the formula 1 / (time_span.days + 1) to calculate the weight. The time_span.days attribute of the timedelta object provides the number of days in the time span between the release date and the present day. By adding 1 to the number of days, the formula ensures that the weight is never zero, even for very recent releases, thus preventing a division by zero error.

###The rationale behind this formula is that the weight diminishes as the time span between the release date and the current day increases. Recent releases receive a higher weight, whereas older releases receive a lower weight. Consequently, when integrating this weighted popularity score with other factors in a recommendation system, recent tracks will exert a greater influence on the final recommendations, reflecting users' potential interest in newer music.

##Next, let's standardize the music features before proceeding further:

In [40]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Track Name        100 non-null    object 
 1   Artists           100 non-null    object 
 2   Album Name        100 non-null    object 
 3   Album ID          100 non-null    object 
 4   Track ID          100 non-null    object 
 5   Popularity        100 non-null    int64  
 6   Release Date      100 non-null    object 
 7   Duration (ms)     100 non-null    int64  
 8   Explicit          100 non-null    bool   
 9   External URLs     100 non-null    object 
 10  Danceability      100 non-null    float64
 11  Energy            100 non-null    float64
 12  Key               100 non-null    int64  
 13  Loudness          100 non-null    float64
 14  Mode              100 non-null    int64  
 15  Speechiness       100 non-null    float64
 16  Acousticness      100 non-null    float64
 17

###Extracting the numerical columns for the model involves isolating the columns in a dataset that contain numerical data. This process is essential for machine learning tasks where the model requires numeric input features. By extracting these columns, we ensure that only relevant numerical data is used for training the model, thereby improving its accuracy and effectiveness in making predictions.

In [41]:
music_df.columns

Index(['Track Name', 'Artists', 'Album Name', 'Album ID', 'Track ID',
       'Popularity', 'Release Date', 'Duration (ms)', 'Explicit',
       'External URLs', 'Danceability', 'Energy', 'Key', 'Loudness', 'Mode',
       'Speechiness', 'Acousticness', 'Instrumentalness', 'Liveness',
       'Valence', 'Tempo'],
      dtype='object')

In [42]:
scaler = MinMaxScaler()

music_features = music_df[['Danceability', 'Energy', 'Key', 'Loudness', 'Mode',
       'Speechiness', 'Acousticness', 'Instrumentalness', 'Liveness',
       'Valence', 'Tempo']].values

music_features_scaled = scaler.fit_transform(music_features)

###We're developing a hybrid recommendation system for music, integrating two distinct approaches. Firstly, we'll recommend music by analyzing its audio features. Secondly, we'll employ a weighted popularity approach to suggest music based on its overall reception and user engagement.

In [43]:
# a function to get content-based recommendations based on music features

def content_based(input_song_name, num_recom = 5):
  if input_song_name not in music_df['Track Name'].values:
    print("The song '{input_song_name}' is not found in the dataset. Please provide a valid song name.")
    return

  # Get the index of the input song in the music DataFrame
  input_song_index = music_df[music_df['Track Name'] == input_song_name].index[0]

  # Calculate the similarity scores based on music features (cosine similarity)
  similarity_scores = cosine_similarity([music_features_scaled[input_song_index]], music_features_scaled)

  # Get the indices of the most similar songs
  similar_song_indices = similarity_scores.argsort()[0][::-1][1 : num_recom + 1]

  # Get the names of the most similar songs based on content-based filtering
  content_based_recommendations = music_df.iloc[similar_song_indices][['Track Name', 'Artists', 'Album Name', 'Release Date', 'Popularity']]

  return content_based_recommendations

Let's dive into the detailed explanation of the `content_based` function:

1. **Input Validation**:
   - The function starts by checking if the input song name provided by the user exists in the dataset (`music_df['Track Name'].values`). This ensures that the recommendation process is based on valid input. If the input song name is not found in the dataset, the function prints a message informing the user to provide a valid song name and then returns `None`.

2. **Index Retrieval**:
   - Assuming the input song name is found in the dataset, the function proceeds to retrieve the index of that song in the music DataFrame (`music_df`). This index will be used to calculate similarity scores with other songs.

3. **Similarity Calculation**:
   - Using cosine similarity, the function calculates how similar the input song is to all other songs in the dataset based on their music features. Cosine similarity measures the cosine of the angle between two vectors and provides a measure of similarity between them. By computing similarity scores for each pair of songs, the function generates a comprehensive overview of their relative similarities, facilitating the identification of top recommendations based on content-based filtering.

4. **Top Recommendations Selection**:
   - After calculating the similarity scores, the function identifies the indices of the most similar songs to the input song. It sorts these indices in descending order of similarity and selects the top `num_recom` recommendations. The `num_recom` parameter allows users to specify how many recommendations they want to receive. It's important to note that the input song itself is excluded from the recommendations to avoid recommending the same song back to the user.

5. **Recommendation Retrieval**:
   - Finally, the function retrieves details of the top recommended songs based on content-based filtering. These details typically include information such as track name, artists, album name, release date, and popularity. By providing this information, the function enables users to explore and discover similar songs that align with their preferences.

In summary, the `content_based` function facilitates the process of generating content-based music recommendations by leveraging music features and similarity calculations. It empowers users to explore new music based on the characteristics of songs they already enjoy.

###We have crafted a function that amalgamates music recommendations derived from weighted popularity with those from content-based filtering, employing a hybrid approach.

In [44]:
def hybrid_recommendations(input_song_name, num_recom=5, alpha=0.5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Get content-based recommendations
    content_based_rec = content_based(input_song_name, num_recom)

    # Check if content_based_rec is a DataFrame
    if not isinstance(content_based_rec, pd.DataFrame):
        print("Content-based recommendations are not available.")
        return

    # Get the popularity score of the input song
    popularity_score = music_df.loc[music_df['Track Name'] == input_song_name, 'Popularity'].values[0]

    # Calculate the weighted popularity score
    weighted_popularity_score = popularity_score * calculate_weighted_popularity(music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0])

    # Combine content-based and popularity-based recommendations based on weighted popularity
    hybrid_recommendations = content_based_rec.copy()
    hybrid_recommendations = pd.concat([hybrid_recommendations, pd.DataFrame({
        'Track Name': [input_song_name],
        'Artists': [music_df.loc[music_df['Track Name'] == input_song_name, 'Artists'].values[0]],
        'Album Name': [music_df.loc[music_df['Track Name'] == input_song_name, 'Album Name'].values[0]],
        'Release Date': [music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0]],
        'Popularity': [weighted_popularity_score]
    })], ignore_index=True)

    # Sort the hybrid recommendations based on weighted popularity score
    hybrid_recommendations = hybrid_recommendations.sort_values(by='Popularity', ascending=False)

    # Remove the input song from the recommendations
    hybrid_recommendations = hybrid_recommendations[hybrid_recommendations['Track Name'] != input_song_name]

    return hybrid_recommendations

Let's break down the function and its components in detail:

1. **Input Validation**:
   - The function begins by validating the input song name provided by the user. It checks if the input song name exists in the dataset (`music_df['Track Name'].values`). If the input song name is not found, it prints a message informing the user to provide a valid song name and returns `None`.

2. **Content-Based Recommendations**:
   - If a valid input song name is provided, the function proceeds to generate content-based recommendations using the `content_based` function. It passes the input song name and the number of desired recommendations (`num_recom`) to the `content_based` function to retrieve recommendations based on music features.

3. **Popularity Score Calculation**:
   - After obtaining content-based recommendations, the function calculates the popularity score of the input song. It retrieves the popularity score from the dataset (`music_df['Popularity']`) based on the input song name.

4. **Weighted Popularity Calculation**:
   - Using a helper function (`calculate_weighted_popularity`), the function computes the weighted popularity score of the input song. This score is derived by multiplying the popularity score of the input song by a weight factor determined based on its release date. The release date of the song is retrieved from the dataset (`music_df['Release Date']`).

5. **Combining Recommendations**:
   - Next, the function combines the content-based recommendations with the input song itself, incorporating its details such as track name, artists, album name, release date, and weighted popularity score. This combined set of recommendations forms the basis of the hybrid approach, integrating both content-based and popularity-based factors in the recommendation process.

6. **Sorting Recommendations**:
   - The hybrid recommendations are sorted based on the weighted popularity score in descending order, ensuring that songs with higher weighted popularity scores are prioritized at the top of the recommendation list.

7. **Removing Input Song**:
   - Finally, the function removes the input song from the recommendation list to avoid redundant suggestions. This step ensures that the input song is not included in the final recommendations provided to the user.

8. **Return**:
   - The function returns the hybrid recommendations, comprising a combination of content-based and popularity-based suggestions, tailored to the user's input song name.

In summary, this function leverages a hybrid approach to generate music recommendations, blending content-based analysis with weighted popularity considerations. By integrating multiple factors in the recommendation process, the function aims to deliver personalized and diverse music suggestions to users, enhancing their listening experience.

In [45]:
input_song_name = "Kina Chir"
recommendations = hybrid_recommendations(input_song_name, num_recom=5)
print(f"Hybrid recommended songs for '{input_song_name}':")
print(recommendations)

Hybrid recommended songs for 'Kina Chir':
    Track Name             Artists   Album Name Release Date  Popularity
4    King Shit               Shubh          Leo   2024-01-05        78.0
0          Her               Shubh          Her   2022-11-30        70.0
3  Showstopper               JERRY  Showstopper   2023-07-21        68.0
1      Rubicon  Prem Dhillon, Rass      Rubicon   2023-08-10        64.0
2  I'm Countin             Harnoor    WITHDRAWS   2024-05-10        49.0


In [46]:
!pip install Dash
!pip install dash_bootstrap_components

In [47]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import dash_bootstrap_components as dbc

# Function to get content-based recommendations based on music features
def content_based(input_song_name, num_recom = 5):
  if input_song_name not in music_df['Track Name'].values:
    print("The song '{input_song_name}' is not found in the dataset. Please provide a valid song name.")
    return

  # Get the index of the input song in the music DataFrame
  input_song_index = music_df[music_df['Track Name'] == input_song_name].index[0]

  # Calculate the similarity scores based on music features (cosine similarity)
  similarity_scores = cosine_similarity([music_features_scaled[input_song_index]], music_features_scaled)

  # Get the indices of the most similar songs
  similar_song_indices = similarity_scores.argsort()[0][::-1][1 : num_recom + 1]

  # Get the names of the most similar songs based on content-based filtering
  content_based_recommendations = music_df.iloc[similar_song_indices][['Track Name', 'Artists', 'Album Name', 'Release Date', 'Popularity']]

  return content_based_recommendations



def hybrid_recommendations(input_song_name, num_recom=5, alpha=0.5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Get content-based recommendations
    content_based_rec = content_based(input_song_name, num_recom)

    # Check if content_based_rec is a DataFrame
    if not isinstance(content_based_rec, pd.DataFrame):
        print("Content-based recommendations are not available.")
        return

    # Get the popularity score of the input song
    popularity_score = music_df.loc[music_df['Track Name'] == input_song_name, 'Popularity'].values[0]

    # Calculate the weighted popularity score
    weighted_popularity_score = popularity_score * calculate_weighted_popularity(music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0])

    # Combine content-based and popularity-based recommendations based on weighted popularity
    hybrid_recommendations = content_based_rec.copy()
    hybrid_recommendations = pd.concat([hybrid_recommendations, pd.DataFrame({
        'Track Name': [input_song_name],
        'Artists': [music_df.loc[music_df['Track Name'] == input_song_name, 'Artists'].values[0]],
        'Album Name': [music_df.loc[music_df['Track Name'] == input_song_name, 'Album Name'].values[0]],
        'Release Date': [music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0]],
        'Popularity': [weighted_popularity_score]
    })], ignore_index=True)

    # Sort the hybrid recommendations based on weighted popularity score
    hybrid_recommendations = hybrid_recommendations.sort_values(by='Popularity', ascending=False)

    # Remove the input song from the recommendations
    hybrid_recommendations = hybrid_recommendations[hybrid_recommendations['Track Name'] != input_song_name]

    return hybrid_recommendations


# Initialize Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Define recommendation types
recommendation_types = [
    {'label': 'Content-based', 'value': 'content_based'},
    {'label': 'Hybrid', 'value': 'hybrid'}
]

# Define app layout
app.layout = dbc.Container([
    html.Div([
        html.H1("Music Recommendation Dashboard", className="display-4 text-center my-4"),

        dbc.Row([
            dbc.Col(
                dcc.Dropdown(
                    id="input-song-dropdown",
                    options=[{'label': song, 'value': song} for song in music_df['Track Name']],
                    placeholder="Select or enter a song",
                    className="form-select",
                    style={'width': '100%'}
                ),
                width=6, xs={'size': 12, 'offset': 0}
            )
        ], className="mb-4"),

        dbc.Row([
            dbc.Col(
                dcc.Dropdown(
                    id="recommendation-type",
                    options=recommendation_types,
                    value='content_based',
                    className="form-select",
                    style={'width': '100%'}
                ),
                width=6, xs={'size': 12, 'offset': 0}
            )
        ], className="mb-4"),

        dbc.Row([
            dbc.Col(
                html.Button(id="submit-button", n_clicks=0, children="Submit", className="btn btn-primary btn-lg btn-block"),
                width=6, xs={'size': 12, 'offset': 0}
            )
        ], className="mb-4"),

        dbc.Row([
            dbc.Col(
                html.Div(id="output-recommendations-container", className="p-3"),
                width=10, xs={'size': 12, 'offset': 0}
            )
        ])
    ], className="bg-light rounded shadow p-4")
])

# Define callback to update recommendations based on user input
@app.callback(
    Output("output-recommendations-container", "children"),
    [Input("submit-button", "n_clicks")],
    [dash.dependencies.State("input-song-dropdown", "value"),
     dash.dependencies.State("recommendation-type", "value")]
)
def update_recommendations(n_clicks, input_song_name, recommendation_type):
    if recommendation_type == 'content_based':
        recommendations = content_based(input_song_name)
    elif recommendation_type == 'hybrid':
        recommendations = hybrid_recommendations(input_song_name)

    # Format recommendations for display
    if recommendations is not None and not recommendations.empty:
        recommendations_table = html.Div([
            html.H3("Recommendations :" , className="text-center mb-4"),
            html.Table([
                html.Tr([
                    html.Th(col, className="bg-secondary text-white") for col in recommendations.columns
                ]),
                html.Tbody([
                    html.Tr([
                        html.Td(recommendations.iloc[i][col]) for col in recommendations.columns
                    ]) for i in range(len(recommendations))
                ])
            ], className="table table-bordered table-striped")
        ])
        return recommendations_table
    else:
        return html.Div("No recommendations available.", className="text-center")

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>